In [1]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import pandas as pd
import os
import json
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
# import graphic objects as go
import plotly.graph_objs as go
import numpy as np
import pandas as pd
import matplotlib
import baycomp

matplotlib.use("pdf")

matplotlib.use("pdf")
os.chdir("/home/isevillano/Github/SHIELD/docs/")


/tmp/ipykernel_2348971/2233138416.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


ModuleNotFoundError: No module named 'baycomp'

# Comparativa Accuracy y Loss

In [5]:
df = pd.DataFrame(columns=['Dataset','Regularization','Percentage', 'Model','Test/Accuracy','Test/Loss'])

for dataset in os.listdir("../results_d/results/"):
    for experiment in os.listdir(f"../results_d/results/{dataset}/"):
        event_acc = EventAccumulator(f"../results_d/results/{dataset}/{experiment}/")
        config_file = json.load(open(f"../results_d/results/{dataset}/{experiment}/config.json"))
        event_acc.Reload()

        Test_Acc = None
        Test_Loss = None
        regularization = ""

        if "Test/Accuracy" in event_acc.Tags()["scalars"]:
            accs = [x.value for x in event_acc.Scalars("Test/Accuracy")]
            Test_Acc = max(accs)

        if "Test/Loss" in event_acc.Tags()["scalars"]:
            losses = [x.value if x.value != np.NAN else np.inf for x in event_acc.Scalars("Test/Loss")]
            Test_Loss = min(losses)

        if config_file["xshield"]:
            regularization = "X-SHIELD"
        if config_file["shield"]:
            regularization = "SHIELD"
        if regularization == "":
            regularization = "Baseline"
        df = pd.concat([df, pd.DataFrame({"Dataset": dataset, 
                                        "Regularization": regularization,
                                        "Model":config_file["pretrained_model"],
                                        "Test/Accuracy": Test_Acc,
                                        "Test/Loss": Test_Loss,
                                        "Percentage": config_file["percentage"]}, index=[0])], ignore_index=True)

df.to_csv("Loss_Acc_results.csv", index=False)


/tmp/ipykernel_2128597/2479459352.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({"Dataset": dataset,
/tmp/ipykernel_2128597/2479459352.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame({"Dataset": dataset,
/tmp/ipykernel_2128597/2479459352.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when 

,Dataset,Regularization,Percentage,Model,Test/Accuracy,Test/Loss
91,OxfordIIITPet,SHIELD,3.0,efficientnet_v2_s,0.924775,NaN
93,OxfordIIITPet,SHIELD,6.0,efficientnet_v2_s,0.922867,NaN


In [5]:
df_latex = df.copy()
df_latex.rename(
    columns={
        "Dataset": "Dataset",
        "Percentage": "Percentage",
        "Model": "Model",
        "Test/Accuracy": "Accuracy",
        "Test/Loss": "Loss",
    },
    inplace=True,
)

df_latex = df_latex[df_latex["Regularization"].isin(["Baseline", "X-SHIELD", "SHIELD"])]
df_latex = df_latex[df_latex["Model"].isin(["efficientnet-b2", "efficientnet_v2_s"])]
df_latex["Accuracy"] = df_latex["Accuracy"].round(4)
df_latex["Loss"] = df_latex["Loss"].round(4)
df_latex = df_latex.sort_values(by=["Dataset", "Model", "Percentage", "Regularization"])

df_latex["Accuracy"] = (
    (df_latex["Accuracy"] * 100)
    .round(2)
    .map(lambda x: f"{x}\%")
    .map(lambda x: x.replace(".", ","))
)
df_latex["Loss"] = (
    df_latex["Loss"].round(4).map(lambda x: f"{x}").map(lambda x: x.replace(".", ","))
)

df_latex[["Accuracy", "Loss"]].to_csv("results.csv", index=False, sep=";")

df_latex["Percentage"] = df_latex["Percentage"].map(lambda x: f"{int(x)}\%")
df_latex = df_latex[
    ["Dataset", "Model", "Percentage", "Regularization", "Accuracy", "Loss"]
]
df_latex["Model"] = df_latex["Model"].map(lambda x: x.replace("_", " "))
df_latex["Model"] = df_latex["Model"].map(lambda x: x.replace("-", " "))

for dataset in df_latex["Dataset"].unique():
    df_ds = df_latex[df_latex["Dataset"] == dataset].drop(columns=["Dataset"])
    # df_ds = df_ds.drop(columns=["Model"])
    # Ordenar df_ds primero por modelo y luego por porcentaje

    df_ds.to_latex(
        f"tables/{dataset}_X-SHIELD.tex",
        index=False,
        escape=False,
        column_format="llccc",
    )


In [6]:
df_bayes = df.copy()
df_bayes = df_bayes[df_bayes["Regularization"].isin(["X-SHIELD", "SHIELD"])]

best_acc = pd.read_csv("best_acc.csv")

df_bayes.merge(best_acc, on=["Model","Dataset","Regularization","Percentage"], how="inner")

# df_bayes = df_bayes[df_bayes["Regularization"].isin(["Baseline", "X-SHIELD", "SHIELD"])]


for model in df_bayes["Model"].unique():
    # Hacer un test de bayes para comparar el Accuracy en test de los modelos
    # Baseline, SHIELD y X-SHIELD
    df_model = df_bayes[df_bayes["Model"] == model]
    
    for metric in ["Test/Accuracy","Test/Loss"]:
        bayesian_list = {}
        for regularization in df_model["Regularization"].unique():
            bayesian_list[regularization] = df_model[df_model["Regularization"] == regularization][metric].values
        
        for reg1 in bayesian_list.keys():
            for reg2 in bayesian_list.keys():
                if reg1 != reg2:
                    val_reg1 = np.array(bayesian_list[reg1])
                    val_reg2 = np.array(bayesian_list[reg2])
                    
                    rope = np.quantile(np.abs(val_reg1 - val_reg2), 0.05)
                    bayesian_test = baycomp.SignTest(val_reg1, val_reg2,rope=rope,nsamples=5000)
                    # cambiar la "/" por "_"
                    metric_name = metric. replace("/","_")
                    bayesian_test.plot(names=[reg1, reg2]).savefig(f"./imagenes/Bayes/{model}_{metric_name}.pdf",
                                                                bbox_inches="tight", 
                                                                dpi=300,
                                                                transparent=True,
                                                                pad_inches=0,
                                                            )
                    matplotlib.pyplot.close("all")

In [7]:
# De cada dataset, cada modelo y cada Recularization, decir que fila es la que mayor accuracy tiene y cual es la que menor loss tiene
df_disp = df.copy()

df_disp = df_disp.sort_values(by=["Dataset", "Model", "Percentage", "Regularization"])

df_disp_acc = df_disp.loc[df_disp.groupby(["Dataset", "Model", "Regularization"])["Test/Accuracy"].idxmax()]
df_disp_acc =df_disp_acc.groupby(["Dataset", "Model","Regularization"]).agg(
    {"Percentage":"first","Test/Accuracy": "max", "Test/Loss": "min"})

df_disp_acc.reset_index().to_csv("best_acc.csv", index=False)

  

In [8]:
# cols tiene los colores default de matplotlib
cols = plt.rcParams['axes.prop_cycle'].by_key()['color'] 
metrics = ["Local_Concordance",
    "Local_Fidelity",
    "Prescriptivity",
    "Conciseness",
    "Robustness"]

bigDf = []
for dataset in os.listdir("../results/"):
    for experiment in os.listdir(f"../results/{dataset}/"):
        # Si existe el archivo "metrics.csv" en el directorio, leerlo
        if os.path.exists(f"../results/{dataset}/{experiment}/metrics.csv"):
            config = json.load(open(f"../results/{dataset}/{experiment}/config.json"))
            df = pd.read_csv(f"../results/{dataset}/{experiment}/metrics.csv")
            
            df["Regularization"] = "SHIELD" if config["shield"] else "X-SHIELD" if config["xshield"] else "Baseline"
            df["Percentage"] = config["percentage"]
            df["Model"] = config["pretrained_model"]
            bigDf.append(df)
        # Si no existe, escribir por pantalla cuál es
        else:
            print(f"No metrics.csv in {experiment} on {dataset} with {config['pretrained_model']} and {config['percentage']}% of hiding.")
    
bigDf = pd.concat(bigDf)
bigDf["Robustness"] = bigDf["robustness"]
bigDf["Conciseness"] = bigDf["conciseness"]
bigDf["Prescriptivity"] = bigDf["prescriptivity"]
bigDf["Local_Fidelity"] = bigDf["local_fidelity"]
bigDf["Local_Concordance"] = bigDf["local_concordance"]
# Eliminar el anterior Experiments.csv
if os.path.exists("Experiments.csv"):
    os.remove("Experiments.csv")
bigDf.drop(columns=["robustness","conciseness","prescriptivity","local_fidelity","local_concordance","name"], inplace=True)
bigDf.dropna(inplace=True)
#bigDf = bigDf[bigDf["index"] <100]
bigDf = bigDf.groupby(["Model","dataset","index","Regularization","Percentage"]).mean().reset_index().to_csv("Experiments_mean.csv", index=False) 

In [9]:
res = pd.read_csv("Experiments_mean.csv")
res = res[res["Regularization"].isin(["SHIELD", "X-SHIELD"])]
res = res[res["Model"].isin(["efficientnet-b2", "efficientnet_v2_s"])]
for metric in metrics:
    lista_figuras = []
    for j,percentage in enumerate(res["Percentage"].unique()):
        res_reg = res[res["Percentage"] == percentage]
        for k, regularization in enumerate(res["Regularization"].unique()): 
            df = res_reg[res_reg["Regularization"] == regularization]
            median = df[metric].quantile(0.5)
            RIQ = df[metric].quantile(0.75) - df[metric].quantile(0.25)
            df_show = df[(df[metric] > median - 1.5*RIQ) & (df[metric] < median + 1.5*RIQ)]
            figura = go.Violin(x = df_show["Model"]+"_"+df_show["dataset"],
                               y = df_show[metric],
                               name = f"{regularization} {percentage}%",
                               box_visible = True,
                               spanmode = "hard",
                               meanline_visible = True,
                               meanline_color = "black",
                               meanline_width = 2,
                               points = False)
            lista_figuras.append(figura)
    fig = go.Figure(data = lista_figuras)
    fig.update_layout(title = metric,
                    title_x = 0.5,
                    violinmode = "group")
    #fig.show(renderer="svg")
    fig.write_image(f"./imagenes/{metric}_violin.pdf",width=1000,height=500)

In [10]:
res = pd.read_csv("Experiments_mean.csv")
best_acc = pd.read_csv("best_acc.csv")
res = res.rename(columns={"dataset":"Dataset"})
res = res.merge(best_acc, on=["Model","Dataset","Regularization","Percentage"], how="inner")
res = res[res["Regularization"].isin(["SHIELD", "X-SHIELD"])]
res = res[res["Model"].isin(["efficientnet-b2", "efficientnet_v2_s"])]
for metric in metrics:
    lista_figuras = []
    for j,regularization in enumerate(res["Regularization"].unique()):
        df = res[res["Regularization"] == regularization]
        median = df[metric].quantile(0.5)
        RIQ = df[metric].quantile(0.75) - df[metric].quantile(0.25)
        df_show = df[(df[metric] > median - 0.5*RIQ) & (df[metric] < median + 0.5*RIQ)]
        figura = go.Violin(x = df_show["Dataset"],
                            y = df_show[metric],
                            name = f"{regularization}",
                            box_visible = True,
                            spanmode = "hard",
                            meanline_visible = True,
                            meanline_color = "black",
                            meanline_width = 2,
                            points = False)
        lista_figuras.append(figura)
        
    fig = go.Figure(data = lista_figuras)
    fig.update_layout(title = metric,
                    title_x = 0.5,
                    violinmode = "group")
    
    fig.write_image(f"./imagenes/{metric}_violin.pdf",width=1000,height=500)



# Test Bayesiano SHIELD vs X-SHIELD

In [11]:


df = pd.read_csv("Experiments_mean.csv")
df = df[df["Regularization"].isin(["SHIELD", "X-SHIELD"])]
df = df[df["Model"].isin(["efficientnet_v2_s"])]


df_shield = df[df["Regularization"] == "SHIELD"]
df_xshield = df[df["Regularization"] == "X-SHIELD"]

# Nos quedamos con la intersección de los dataset donde "Model", "dataset", "Percentage" y "index" son iguales
df_instersection = df_shield.merge(df_xshield, on=["Model", "dataset", "Percentage", "index"], suffixes=["_shield", "_xshield"])


# Para cada métrica, comparamos los resultados de SHIELD y X-SHIELD con un test de Bayes con un rope igual al
# 5% de la diferencia de las medias
for percentage in df_instersection["Percentage"].unique():
    df_percentage = df_instersection[df_instersection["Percentage"] == percentage]
    for metric in metrics:
        rope = np.quantile(np.abs(df_percentage[metric+"_shield"] - df_percentage[metric+"_xshield"]), 0.20)
        SHIELD = df_percentage[metric+"_shield"]
        XSHIELD = df_percentage[metric+"_xshield"]
        bayes_test = baycomp.SignTest(SHIELD, XSHIELD,rope=rope,nsamples=10000)
        
        bayes_test.plot(names=["SHIELD", "XSHIELD"]).savefig(f"./imagenes/Bayes/{metric}_{percentage}.pdf", bbox_inches="tight", dpi=300,
                                                            transparent=True,
                                                            pad_inches=0,
                                                            )
        # Cerrar la figura para que no se acumulen
        matplotlib.pyplot.close("all")


    

# Mejor X-SHIELD vs mejor SHIELD para cada dataset

In [12]:
df = pd.read_csv("Experiments_mean.csv")

best_acc = pd.read_csv("best_acc.csv")
df = df.rename(columns={"dataset":"Dataset"})
df = df.merge(best_acc, on=["Model","Dataset","Regularization","Percentage"], how="inner")
df = df[df["Regularization"].isin(["SHIELD", "X-SHIELD"])]
df = df[df["Model"].isin(["efficientnet_v2_s"])]


df_shield = df[df["Regularization"] == "SHIELD"]
df_xshield = df[df["Regularization"] == "X-SHIELD"]

for metric in metrics:
    rope = np.quantile(np.abs(np.array(df_shield[metric]) - np.array(df_xshield[metric])), 0.20)
    
    SHIELD = np.array(df_shield[metric])
    XSHIELD = np.array(df_xshield[metric])
    bayes_test = baycomp.SignTest(SHIELD, XSHIELD,rope=rope,nsamples=10000)
    bayes_test.plot(names=["SHIELD", "XSHIELD"]).savefig(f"./imagenes/Bayes/{metric}_best.pdf", bbox_inches="tight", dpi=300,
                                                        transparent=True,
                                                        pad_inches=0,
                                                        )
    matplotlib.pyplot.close("all")

# Comparativa mejor X-SHIELD vs mejor SHIELD para cada dataset en train y test

In [35]:
df_best = pd.read_csv("best_acc.csv")
df_best = df_best[df_best["Model"].isin(["efficientnet_v2_s"])]
df_best = df_best[df_best["Regularization"].isin(["SHIELD", "X-SHIELD"])].reset_index(drop=True)

for dataset in df_best["Dataset"].unique():
    df_datasets = df_best[df_best["Dataset"] == dataset]
    
    # Añadir una figura cuya x es el número de iteraciones y cuya y es el valor de la métrica 
    # y el color es el tipo de regularización
    for metric in ["Val/Accuracy","Val/Loss"]:
        fig = go.Figure()
        for row in df_datasets.iterrows():
            row = row[1]
            if row["Regularization"] == "Baseline":
                event_ac_name = f"../results_d/results/{row['Dataset']}/{row['Regularization']}_{row['Model']}/"
            else:
                event_ac_name = f"../results_d/results/{row['Dataset']}/{row['Regularization']}_{row['Model']}_{row['Percentage']}/"
            event_acc = EventAccumulator(event_ac_name)
            event_acc.Reload()
            metric_evolution = np.array([x.value for x in event_acc.Scalars(metric)])
            # Cambia los valores extremos por la mediana de la ventana de 5 centrada en el valor
            
            RIQ = np.quantile(metric_evolution, 0.75) - np.quantile(metric_evolution, 0.25)
            up_limit = np.quantile(metric_evolution, 0.75) + 1*RIQ
            low_limit = np.quantile(metric_evolution, 0.25) - 1*RIQ
            for i in range(2,len(metric_evolution)-2):
                if metric_evolution[i] > up_limit or metric_evolution[i] < low_limit:
                    metric_evolution[i] = np.median(metric_evolution[i:i+5])
            
            # Suaviza los puntos extremos
            metric_evolution = np.convolve(metric_evolution, np.ones(3)/3, mode='valid')
            
            color_metric =cols[0]
            if row["Regularization"] == "X-SHIELD":
                color_metric = cols[1]
            if row["Regularization"] == "Baseline":
                color_metric = cols[2]
            fig.add_trace(go.Scatter(x = np.array(range(1, len(metric_evolution)+1)),
                                    y = metric_evolution,
                                    mode = "lines",
                                    name = f"{row['Regularization']} {row['Percentage']}%",
                                    #color = color_metric,
                                    line = dict(color = cols[0] if row["Regularization"] == "SHIELD" else cols[1] if row["Regularization"] == "X-SHIELD" else cols[2] )),
                        )
            
        fig.update_layout(title = f"{dataset} {metric
                        title_x = 0.5,
                        xaxis_title = "Epochs",
                        yaxis_title = metric)
        fig.show()
        fig.write_image(f"./imagenes/{dataset}_{metric.replace('/','_' )}.pdf",width=1000,height=500)
        